<b>Pandas之数据分组和聚合</b>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'key1':['a','a','b','b','a'],'key2':['one','two','one','two','one'],'data1':np.random.randn(5),'data2':np.random.randn(5)})
df

,data1,data2,key1,key2
0,-1.446650,-0.594725,a,one
1,0.624239,-0.778013,a,two
2,1.102340,-0.044075,b,one
3,0.926252,0.137297,b,two
4,-0.370969,0.878950,a,one


<b>数据分组（groupby）</b>

.groupby(by=None, axis=0, level=None, as_index=True, sort=True, group_keys=True, squeeze=False)方法作用于一条轴向上，并接受一个分组键（by）参数来给调用者分组。分组键可以是Series 或列表，要求其长度与待分组的轴一致；也可以是映射函数、字典甚至数组的某条列名（字符串），但这些参数类型都只是快捷方式，其最终仍要用于生成一组用于拆分对象的值。

In [3]:
grouped = df.groupby(df['key1'])
grouped

这里使用 df['key1'] 做了分组键，即按 a 和 b 进行分组。但实际分组键并不需要与数组对象之间存在联系，只要长度相同即可，使用数组的列只是图方便。上例中如果使用 [1,1,2,2,3] 这样的列表做分组键的话，结果与 df['key1'] 是相同的。  
groupby 方法的调用本身并不涉及运算，因此速度很快。而在操作这个 grouped 对象的时候，还是将其看成一个保存了实际数据的对象比较方便。比如我们可以直接对其应用很多方法，或索引切片：

In [4]:
grouped.mean()

,data1,data2
key1,,
a,-0.397793,-0.164596
b,1.014296,0.046611


上例中没有显示 key2 列，是因为其值不是数字类型，被 mean() 方法自动忽视了。当想要只看某一（些）列的时候，可以通过索引来实现，在 groupby 方法调用前后均可

In [5]:
df.groupby(df['key1'])['data1'].mean()

key1
a   -0.397793
b    1.014296
Name: data1, dtype: float64

如果分组键使用的是多个数组，就会得到一个层次化索引的结果：

In [6]:
df.groupby([df['key1'],df['key2']]).mean()

data1     data2
key1 key2                    
a    one  -0.908810  0.142112
     two   0.624239 -0.778013
b    one   1.102340 -0.044075
     two   0.926252  0.137297

最后，可以使用 GroupBy 对象（不论是 DataFrameGroupBy 还是 SeriesGroupBy）的 .size() 方法查看分组大小：

In [7]:
grouped.size()

key1
a    3
b    2
dtype: int64

GroupBy 对象是可以通过 for 循环迭代的，可以产生一组二元组，分别为分组名和组内数据。下面是一个多重分组键的情况：

In [8]:
for i,j in df.groupby([df['key1'],df['key2']]):
    print(i)
    print('-----------')
    print(j)

('a', 'one')
-----------
      data1     data2 key1 key2
0 -1.446650 -0.594725    a  one
4 -0.370969  0.878950    a  one
('a', 'two')
-----------
      data1     data2 key1 key2
1  0.624239 -0.778013    a  two
('b', 'one')
-----------
     data1     data2 key1 key2
2  1.10234 -0.044075    b  one
('b', 'two')
-----------
      data1     data2 key1 key2
3  0.926252  0.137297    b  two


In [9]:
list(df.groupby([df['key1'],df['key2']]))

[(('a', 'one'),       data1     data2 key1 key2
  0 -1.446650 -0.594725    a  one
  4 -0.370969  0.878950    a  one),
 (('a', 'two'),       data1     data2 key1 key2
  1  0.624239 -0.778013    a  two),
 (('b', 'one'),      data1     data2 key1 key2
  2  1.10234 -0.044075    b  one),
 (('b', 'two'),       data1     data2 key1 key2
  3  0.926252  0.137297    b  two)]

In [10]:
dict(list(df.groupby([df['key1'],df['key2']])))

{('a', 'one'):       data1     data2 key1 key2
 0 -1.446650 -0.594725    a  one
 4 -0.370969  0.878950    a  one, ('a', 'two'):       data1     data2 key1 key2
 1  0.624239 -0.778013    a  two, ('b', 'one'):      data1     data2 key1 key2
 2  1.10234 -0.044075    b  one, ('b', 'two'):       data1     data2 key1 key2
 3  0.926252  0.137297    b  two}

<b>使用字典或Series作分组键</b>

这两种参数需要提供一种从行（列）名到组名的映射关系。

In [11]:
df2 = pd.DataFrame({'data1':[2, 2, 1, 4, 3],'data2':[1, 3, 5, 7, 9],'data3':[2, 4, 6, 8, 10]}, index=['a','b','c','d','e'])
df2

,data1,data2,data3
a,2,1,2
b,2,3,4
c,1,5,6
d,4,7,8
e,3,9,10


In [12]:
mapping = {'a': 'bule', 'b': 'bule', 'c': 'bule', 'd': 'red', 'e': 'red'}
df2.groupby(mapping,as_index=True,axis=0).sum()

,data1,data2,data3
bule,5,9,12
red,7,16,18


In [13]:
mapping2 = {'data1': 'one', 'data2': 'two', 'data3': 'one'}
df2.groupby(mapping2,axis=1).sum()

,one,two
a,4,1
b,6,3
c,7,5
d,12,7
e,13,9


<b>通过函数进行分组</b>

函数的作用有些类似于字典，或者说这些奇怪的分组键都类似于字典——利用某种映射关系将待分组的轴转化为一个等长的由分组名组成的序列。

如果说行列名是作为索引传递给字典以获取组名的话，那么在函数分组键中，行列名就会作为参数传递给函数。这便是你需要提供的函数类型：

In [14]:
df2.groupby(lambda x:'small' if x<='c' else 'big').sum()

,data1,data2,data3
big,7,16,18
small,5,9,12


<b>根据索引级别分组</b>

当根据高级别索引来分组的时候，参数就不再是 by=None 了，而要换成 level=None，值可以是索引级别的编号或名称：

In [15]:
index = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],[1, 3, 5, 1, 3]], names=['city', 'tenor'])
df.index = index
df

data1     data2 key1 key2
city tenor                              
US   1     -1.446650 -0.594725    a  one
     3      0.624239 -0.778013    a  two
     5      1.102340 -0.044075    b  one
JP   1      0.926252  0.137297    b  two
     3     -0.370969  0.878950    a  one

In [16]:
df.groupby(level='city').sum()

,data1,data2
city,,
JP,0.555283,1.016248
US,0.279929,-1.416814


In [17]:
df.groupby(level=0).sum()

,data1,data2
city,,
JP,0.555283,1.016248
US,0.279929,-1.416814


<b>数据聚合（Aggregation）</b>

数据聚合，指的是任何能够从数组产生标量值的数据转换过程。你也可以简单地将其理解为统计计算，如 mean(), sum(), max() 等。

数据聚合本身与分组并没有直接关系，在任何一列（行）或全部列（行）上都可以进行。不过当这种运算被应用在分组数据上的时候，结果可能会变得更有意义。

对于 GroupBy 对象可以应用的聚合运算包括：

　已经内置的方法，如 sum()， mean() 等  
　Series 的方法，如 quantile() 等  
　自定义的聚合函数，通过传入 GroupBy.aggregate() 或 GroupBy.agg() 来实现  

其中自定义函数的参数应当为一个数组类型，即 GroupBy 对象迭代出的元组的第二个元素。如

In [18]:
df.index=range(0,len(df))
df

,data1,data2,key1,key2
0,-1.446650,-0.594725,a,one
1,0.624239,-0.778013,a,two
2,1.102340,-0.044075,b,one
3,0.926252,0.137297,b,two
4,-0.370969,0.878950,a,one


In [19]:
df.groupby('key1')['data1','data2'].agg(lambda arr: arr.max() - arr.min())

,data1,data2
key1,,
a,2.070890,1.656964
b,0.176087,0.181372


但是自定义聚合运算的效率其实很慢。

<b>面向列的多函数应用</b>

如果想对相同列应用多个函数，只需给 agg() 传入一个函数列表即可。

In [20]:
flists = ['max','min']
df.groupby('key1')['data1','data2'].agg(flists)

data1               data2          
           max       min       max       min
key1                                        
a     0.624239 -1.446650  0.878950 -0.778013
b     1.102340  0.926252  0.137297 -0.044075

可以对计算结果进行重命名。

In [21]:
flists = [('最大值','max'),('最小值','min')]
df.groupby('key1')['data1','data2'].agg(flists)

data1               data2          
           最大值       最小值       最大值       最小值
key1                                        
a     0.624239 -1.446650  0.878950 -0.778013
b     1.102340  0.926252  0.137297 -0.044075

如果想对想对不同列应用不同函数，只需给agg()传入一个从列名映射到函数名的字典。

In [22]:
ftuples = {'data1':'max','data2':['max','mean']}
df.groupby('key1')['data1','data2'].agg(ftuples)

data1     data2          
           max       max      mean
key1                              
a     0.624239  0.878950 -0.164596
b     1.102340  0.137297  0.046611

<b>分组级运算和转换</b>

聚合只是分组运算的一种，更多种类的分组运算可以通过 .transform() 和 apply() 方法实现。

In [23]:
df

,data1,data2,key1,key2
0,-1.446650,-0.594725,a,one
1,0.624239,-0.778013,a,two
2,1.102340,-0.044075,b,one
3,0.926252,0.137297,b,two
4,-0.370969,0.878950,a,one


<b>transform</b>

前面进行聚合运算的时候，得到的结果是一个以分组名为 index 的结果对象。虽然可以指定as_index=False,但是得到的索引也并不是元数据的index。如果我们想使用原数组的 index 的话，就需要进行 merge 转换。  
transform(func, *args, **kwargs) 方法简化了这个过程，它会把 func 参数应用到所有分组，然后把结果放置到原数组的 index 上（如果结果是一个标量，就进行广播）

In [24]:
df.groupby('key1',as_index=False).agg('max')

,key1,data1,data2,key2
0,a,0.624239,0.878950,two
1,b,1.102340,0.137297,two


In [25]:
df.groupby('key1').transform('max').add_prefix('groupby_key1_max_')

,groupby_key1_max_data1,groupby_key1_max_data2,groupby_key1_max_key2
0,0.624239,0.878950,two
1,0.624239,0.878950,two
2,1.102340,0.137297,two
3,1.102340,0.137297,two
4,0.624239,0.878950,two


<b>apply</b>

apply(func, *args, **kwargs) 会将待处理的对象拆分成多个片段，然后对各片段调用传入的函数，最后尝试用 pd.concat() 把结果组合起来。func 的返回值可以是 pandas 对象或标量，并且数组对象的大小不限。

In [27]:
df.groupby('key1').apply(np.max)
# grouped.apply()

,data1,data2,key1,key2
key1,,,,
a,0.624239,0.878950,a,two
b,1.102340,0.137297,b,two


apply方法可以返回任意的结果，所以实际应用时应该考虑如何去限制发挥apply方法。

In [30]:
def foo(arr,num=6):
    df = pd.DataFrame(np.arange(num).reshape(2,3), index=['row1','row2'])
    return df
df.groupby('key1').apply(foo)

0  1  2
key1              
a    row1  0  1  2
     row2  3  4  5
b    row1  0  1  2
     row2  3  4  5

ref：http://my.oschina.net/lionets/blog/280332